## Lab 2 - Applied Machine Learning (DT4031, DT4033) VT26

### Instructions
##### Flu Shot Learning: Predicting H1N1 and Seasonal Flu Vaccines
Vaccination is one of the key measures in global health to reduce spread of infection, diseases and death, as well as decrease both costs and workload in healthcare. One flu known for its annual vaccination is the seasonal flu, caused by Influenza A. This type of flu is particularly persistent and mutating, which makes it necessary to keep developing the existing vaccine in order to give some protection to people at risk. Another known influenza is the H1N1 flu, also known as the swine flu. In 2009, this flu was declared to be a pandemic by the World Health Organization due to its widespread infections and high rates of mortality. In addition to the normal seasonal flu vaccine, a separate vaccine was developed for this particular influenza with improved efficiency of reducing the rates and seriousness of the illness. Millions of doses were given across populations in the world in order to overcome the pandemic. 

This is a multilabel classification task where your goal is to predict how likely individuals are to receive their H1N1 and seasonal flu vaccines using information they shared about their backgrounds, opinions, and health behaviours. Specifically, you will be predicting two probabilities: one for h1n1_vaccine and one for seasonal_vaccine. This will be done for multiple classifiers, which results you will then compare and analyze. 

The data comes from the National 2009 H1N1 Flu Survey (NHFS), which was a list-assisted random-digit-dialing telephone survey, designed to monitor influenza immunization coverage in the 2009-10 season. The target population was people older than 6 months in the United States. Each row in the dataset represents one person who responded to the survey. 

You can read more about the dataset and the challenge this lab is based upon here: https://www.drivendata.org/competitions/66/flu-shot-learning/page/211/  

### Submission 
* Submit all your code as a single Jupyter notebook.
* Structured report that includes Introduction, Method, Results, Discussion. The results should contain the evaluation, including figures comparing the different classifiers. The report should NOT contain any code. An analysis should be performed on the results, comparing the different classifiers to each other and the effect of optimizing some hyperparameters. The discussion should additionally contain contemplations guided by the questions in the lab, regarding data exploration, method choices, effects of feature selections, and evaluation. 


In [ ]:
!pip install missingno

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import missingno as msno 
import seaborn as sns



### Part 1

In [ ]:
df_features = pd.read_csv("features.csv")
df_labels = pd.read_csv("labels.csv")
df = pd.concat([df_features, df_labels], axis = 1)
df


#### Data description 
Make sure you spend some time understanding the data. Answer the following questions (for example using figures): 
* In the data, how many people took the H1N1 vaccine vs the seasonal vaccine? How many didn't take it? 
* Which columns hold categorical data and which hold numerical data (including binary)? 
* Is there an imbalance in the data, considering the labels? 
* Choose 3 columns and check if they have any boundary violations by comparing the values to the description of what values it should contain (given in the link above). 

In [ ]:
df.describe()

In [ ]:
#Add code here to explore the dataset and answer the given questions 

#### Some quick preprocessing
When we look at df.describe() we can see that the count between the different features vary, which indicates that there are missing values we need to take care of. Let's investigate.  

In [ ]:
df.isnull().sum()

In [ ]:
msno.matrix(df)
plt.show()

In [ ]:
missing_values_df = df.isna().sum()

features = []
missing = []

for x in missing_values_df:
    missing.append((x / df.shape[0]) * 100) 
    
for feature in df:
    features.append(feature)
    
data = {'Features': features, 'Missing': missing}

missing_values = pd.DataFrame(data)
missing_values = missing_values.sort_values(by=["Missing"], ascending=[False])
sns.set(rc={'figure.figsize':(6,10)})
missing_plot = sns.barplot(y="Features", x ='Missing', data=missing_values)

plt.xlabel("Missing values (%)", fontsize=20)
plt.ylabel("Features", fontsize=20)
plt.tight_layout()
plt.show()

Looking at the information above, decide which features have a very high amount of missing values. What potential problem could arise from including these features in the dataset during model training? Consider the impact of removing these based on your reasoning, decide which features are most appropriate to exclude. This could be none, some, or all of them.


 

In [ ]:
df = df.drop(columns=[]) #add the chosen features in the brackets

Now we fix the remaining missing values. We will impute numerical and categorical features seperately. For categorical features we will for simplicity use the most frequent values to impute with. 

Before we impute we need to split our dataset. This is due to not wanting to leak any data while imputing. If we imputed with the test data, the training data could be influenced and the test data would thereby not be unseen. Therefore, we will fit the imputer to the training data and use it to transform our test and validation dataset. 

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=["h1n1_vaccine", "seasonal_vaccine"])
y = df[["h1n1_vaccine", "seasonal_vaccine"]]


X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=, random_state=42) #add size of split
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=, random_state=42) #add size of split



print("Missing values in X_train:", X_train.isna().sum().sum())
print("Missing values in X_validation:", X_val.isna().sum().sum()) 
print("Missing values in X_test:", X_test.isna().sum().sum()) 
print("Missing values in y_train:", y_train.isna().sum().sum()) #should already be 0
print("Missing values in y_validation:", y_val.isna().sum().sum()) #should already be 0
print("Missing values in y_test:", y_test.isna().sum().sum()) #should already be 0



In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer


num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns #numerical columns
cat_cols = X_train.select_dtypes(include=["object"]).columns #categorical columns

#numerical
num_imputer = IterativeImputer(random_state=42)
X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
X_val[num_cols] = num_imputer.transform(X_val[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

#categorical
cat_imputer = SimpleImputer(strategy="most_frequent")
X_train[cat_cols] = cat_imputer.fit_transform(X_train[cat_cols])
X_val[cat_cols] = cat_imputer.transform(X_val[cat_cols])
X_test[cat_cols] = cat_imputer.transform(X_test[cat_cols])


In [ ]:
#these should be 0 if imputed correctly
print("Missing values in X_train:", X_train.isna().sum().sum())
print("Missing values in X_validation:", X_val.isna().sum().sum()) 
print("Missing values in X_test:", X_test.isna().sum().sum()) 
print("Missing values in y_train:", y_train.isna().sum().sum())
print("Missing values in y_validation:", y_val.isna().sum().sum()) 
print("Missing values in y_test:", y_test.isna().sum().sum()) 

#### Feature selection

When it comes to working with real-world datasets, they are often messy and can contain hundreds or even thousands of features for every instance. This can become problematic as high dimensionality can negatively affect model performance (see the curse of dimensionality) while many features may contribute little to the model's decision-making process. 

Feature selection is a data pre-processing method used to reduce the number of features in a dataset by identifying redundant and irrelevant features. This can help the model to, among other things, reduce the risk of overfitting, decrease computational requirements and facilitate interpretability of results. 

There are multiple approaches to feature selection, ranging from automated algorithms to conceptual methods based on common sense, domain knowledge and data-quality considerations. You already did some feature selection when handling features with a lot of missing values. 

Consider all features in the dataset in the context of the task: a multilabel classification of H1N1 and sesonal flu vaccination. Are there any features that could give one label more direct information than the other? How might this affect model learning, and which features, if any, might you consider removing to ensure the model receives comparable information for both labels? 

In [ ]:
df.columns

In [ ]:

cols_to_drop = [] #add potential features that you wish to drop based on your reasoning

X_train = X_train.drop(columns=cols_to_drop)
X_val = X_val.drop(columns=cols_to_drop)
X_test = X_test.drop(columns=cols_to_drop)


In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape) #make sure it matches 

Now that we have relevant candidates for features, we will try an algorithm feature selection technique called SelectKBest. SelectKBest score each feature independently against the target variable using a statistical test, and then selects the top K features with the highest scores. This means that the features are based on the scoring function. 
Read more about it here: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html. 

As the task is multilabel, we will use the f_classif scoring function seperately for each target variable. For each feature, we will calculate its score for both H1N1 and seasonal vaccines, and then compute an average score to identify features that are important for both targets. This requires numerical values, so first we will be encoding these. Please note that using the average, we are considering which features are the most important for both targets at once. If we want to achieve better performance for the task, getting the most important features for each target would probably yield better results. However, for simplicity, we are considering both at the same time. If you are interested and feel confident in changing more of the code, you can try making separate models and feature selections for the different targets, but this is not a requirement. 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns #numerical columns
cat_cols = X_train.select_dtypes(include=["object"]).columns #categorical columns

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_encoded = encoder.fit_transform(X_train[cat_cols])
X_val_encoded = encoder.transform(X_val[cat_cols])
X_test_encoded = encoder.transform(X_test[cat_cols])

X_train_cat = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(cat_cols)).reset_index(drop=True)
X_val_cat = pd.DataFrame(X_val_encoded, columns=encoder.get_feature_names_out(cat_cols)).reset_index(drop=True)
X_test_cat = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(cat_cols)).reset_index(drop=True)

X_train = pd.concat([X_train[num_cols].reset_index(drop=True), X_train_cat], axis=1)
X_val= pd.concat([X_val[num_cols].reset_index(drop=True), X_val_cat], axis=1)
X_test = pd.concat([X_test[num_cols].reset_index(drop=True), X_test_cat], axis=1)

print(X_train.shape, X_val.shape, X_test.shape) #the amount of columns should now have increased


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

selector_h1n1 = SelectKBest(score_func=f_classif, k='all')
selector_h1n1.fit(X_train, y_train["h1n1_vaccine"])
h1n1_scores = pd.DataFrame({"Feature": X_train.columns,"F_score_h1n1": selector_h1n1.scores_})

selector_seasonal = SelectKBest(score_func=f_classif, k='all')
selector_seasonal.fit(X_train, y_train["seasonal_vaccine"])
seasonal_scores = pd.DataFrame({"Feature": X_train.columns,"F_score_seasonal": selector_seasonal.scores_})

feature_scores = pd.merge(h1n1_scores, seasonal_scores, on="Feature")
feature_scores["Average_score"] = feature_scores[["F_score_h1n1","F_score_seasonal"]].mean(axis=1)

feature_scores = feature_scores.sort_values(by="Average_score", ascending=False).reset_index(drop=True)

feature_scores.head(10) #change here to see more or less features


Looking at the feature scores, do you think they make sense? Why/why not? Then choose a k below. As this is multilabel, we will choose the features based on their average scores. Normally you could use the transform method for this (non-multilabel), but here we do it a bit more manual. 

In [ ]:
top_k = #the number of features you want to keep based on average score
top_features = feature_scores["Feature"].head(top_k).tolist()
print("Top features by average score:", list(top_features))

X_train = X_train[top_features]
X_val = X_val[top_features]
X_test = X_test[top_features]

print(X_train.shape, X_val.shape, X_test.shape) #should match your chosen features 

### Part 2 
#### Training

Finally training time! We've implemented two models for you below. The code uses a MultiOutputClassifier (https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) that fits one classifier per target. If you want to, you can instead train one model for each target. Model number 1 is a decision tree: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


base_tree = DecisionTreeClassifier(random_state=42)
multi_tree = MultiOutputClassifier(base_tree)

multi_tree.fit(X_train, y_train)

y_val_pred = multi_tree.predict(X_val)

print("Accuracy (val): ", accuracy_score(y_val, y_val_pred))

print("Validation Metrics")
print(classification_report(y_val, y_val_pred, target_names=["H1N1", "Seasonal"], zero_division=0))


Most often models don't perform well immediately but require some hyperparameter optimization to reach a more satisfactory result. You can check the individual models parameters to check what you can try to optimize. For the decision tree, the depth of tree can be crucial. Try picking a suitable range of depth for better performance. 

Note that since we use a MultiOutputClassifier, the accuracy shows over both target variables. This means we will hence optimize for both target variables at once, and only get one set of hyperparameters. Most likely, this is not the most optimal hyperparameters for the target variables individually. If you want, you can change this and train two models individually, and in extension probably receive better performance. 

In [ ]:
max_depth = []
accuracies = []
for i in range(): # pick a suitable range
    decision_tree_base = DecisionTreeClassifier(max_depth=i)
    decision_tree_model = MultiOutputClassifier(decision_tree_base)
    decision_tree_model.fit(X_train, y_train)
    decision_tree_pred = decision_tree_model.predict(X_val)
    accuracy = round(accuracy_score(y_val, decision_tree_pred), 5)
    max_depth.append(i)
    accuracies.append(accuracy)

decision_tree_hyperparameter_tuning_df = pd.DataFrame({'max_depth': max_depth, 'accuracy': accuracies})
decision_tree_hyperparameter_tuning_df

In [ ]:
#choose your optimal hyperparameters for a final model 
decision_tree = DecisionTreeClassifier(max_depth=) 
optimal_decision_tree_model = MultiOutputClassifier(decision_tree) 
optimal_decision_tree_model.fit(X_train, y_train)
optimal_decision_tree_pred = optimal_decision_tree_model.predict(X_test) #here we predict the hold-out dataset
optimal_decision_tree_pred_prob = optimal_decision_tree_model.predict_proba(X_test)
optimal_decision_tree_y_preds = pd.DataFrame({"h1n1_vaccine": optimal_decision_tree_pred_prob[0][:, 1],"seasonal_vaccine": optimal_decision_tree_pred_prob[1][:, 1],},)


Model number two is an ensemble learning model called AdaBoost. Read more about it here: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

basic_ada_boost_model = AdaBoostClassifier(algorithm="SAMME")
basic_ada_boost_multi_model= MultiOutputClassifier(basic_ada_boost_model)
basic_ada_boost_multi_model=basic_ada_boost_multi_model.fit(X_train, y_train)
basic_ada_boost_y_pred = basic_ada_boost_multi_model.predict(X_val)

print("Accuracy (val): ", accuracy_score(y_val, basic_ada_boost_y_pred))

print("Validation Metrics")
print(classification_report(y_val, basic_ada_boost_y_pred, target_names=["H1N1", "Seasonal"], zero_division=0))


Similarly, we will try some basic optimization here. Choose which numbers of estimators you'd like to try and choose the one that works the best. 


In [ ]:
accuracies_test = []
accuracies_train = []

n_estimators = [] #choose which values to try 

for est in n_estimators:     
    ada_boost_model = AdaBoostClassifier(n_estimators=est, algorithm="SAMME")
    ada_boost_multi_model= MultiOutputClassifier(ada_boost_model)
    ada_boost_multi_model=ada_boost_multi_model.fit(X_train, y_train)
    ada_boost_y_pred = ada_boost_multi_model.predict(X_val)
    ada_boost_y_train_pred = ada_boost_multi_model.predict(X_train)
    accuracy_test = accuracy_score(y_val, ada_boost_y_pred)
    accuracy_train = accuracy_score(y_train, ada_boost_y_train_pred)
    accuracies_test.append(accuracy_test)
    accuracies_train.append(accuracy_train)
    

ada_hpo = pd.DataFrame({'n_estimators': n_estimators, 'Accuracy train': accuracies_train, 'Accuracy val': accuracies_test})
ada_hpo

In [ ]:
optimal_nestimators =  #choose your value from last cell
optimal_ada_boost_model = AdaBoostClassifier(n_estimators = optimal_nestimators, algorithm = "SAMME")
optimal_ada_boost_multi_model= MultiOutputClassifier(optimal_ada_boost_model)
optimal_ada_boost_multi_model=optimal_ada_boost_multi_model.fit(X_train, y_train)
optimal_ada_boost_y_pred = optimal_ada_boost_multi_model.predict(X_test) #predict hold-out data
optimal_ada_boost_pred_prob = optimal_ada_boost_multi_model.predict_proba(X_test)
optimal_ada_boost_y_preds = pd.DataFrame(
    {
        "h1n1_vaccine": optimal_ada_boost_pred_prob[0][:, 1],
        "seasonal_vaccine": optimal_ada_boost_pred_prob[1][:, 1],
    },
)

Now it's your turn! Choose a model and make some hyperparameter optimization and prepare it for comparison. You can find more classifiers for example from lectures, in Chapter 7 from "Hands-On Machine Learning with Scikit-Learn, Keras, and Tensorflow", or in the documentation from scikit-learn. 

In [ ]:
#add your code here 

#### Evaluation

Finally, we will compare the models based on hold-out data and look on how to evaluate the models. So far we've created optimal models and predicted on the hold-out dataset using those. Let's look at the metrics. When analyzing, consider what metrics are the most useful in the context of the task. Can the importance of different metrics differ between different tasks? Give an example. 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score,roc_auc_score

In [ ]:
def evaluate_multilabel_model(model_name,y_true,y_pred,y_pred_proba,label_names=("h1n1_vaccine", "seasonal_vaccine"),):
    metrics = {
        "model": model_name,
        "accuracy": accuracy_score(y_true, y_pred),
        "precision_macro": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "recall_macro": recall_score(y_true, y_pred, average="macro", zero_division=0),
        "f1_macro": f1_score(y_true, y_pred, average="macro", zero_division=0),
    }

    y_proba = pd.DataFrame(
        {
            label: y_pred_proba[i][:, 1]
            for i, label in enumerate(label_names)
        }).values

    metrics["roc_auc_macro"] = roc_auc_score(y_true,y_proba,average="macro",multi_class="ovo")

    return pd.DataFrame([metrics])

In [ ]:
results = []

results.append(
    evaluate_multilabel_model(
        model_name="Decision Tree",
        y_true=y_test,
        y_pred=optimal_decision_tree_pred,
        y_pred_proba=optimal_decision_tree_pred_prob,
    )
)

results.append(
    evaluate_multilabel_model(
        model_name="AdaBoost",
        y_true=y_test,
        y_pred=optimal_ada_boost_y_pred,
        y_pred_proba=optimal_ada_boost_pred_prob,
    )
)


#add metrics for your own model here 

metrics_df = pd.concat(results, ignore_index=True)
metrics_df


If we want to visualise this more clearly, we can look at the confusion matrices. This will show us what the model predicts in comparison to their true labels in a classification task. We detect true positives(TP, model predicted correct positive class), true negatives (TN, model predicted correct negative class), false positives(FP, model predicts a positive class when in reality it's not), and false negatives (model predicts a negative class when it should be positive). 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix


def plot_multilabel_confusion_matrices(model_cm_dict, labels=("H1N1", "Seasonal"), cmap="Blues"):
    n_models = len(model_cm_dict)
    n_rows = n_models
    n_cols = len(labels)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 4 * n_rows))

    if n_rows == 1:
        axes = axes.reshape(1, -1)

    for i, (model_name, cms) in enumerate(model_cm_dict.items()):
        for j, cm in enumerate(cms):
            ax = axes[i, j]
            sns.heatmap(cm, annot=True, fmt="d", cmap=cmap, ax=ax)
            ax.set_title(f"{model_name} - {labels[j]}", fontsize=14)
            ax.set_xlabel("Predicted value", fontsize=10)
            ax.set_ylabel("True value", fontsize=10)
    
    fig.tight_layout()
    plt.show()


optimal_decision_tree_cm = multilabel_confusion_matrix(
    y_true=y_test,
    y_pred=optimal_decision_tree_pred
)

optimal_adaboost_cm = multilabel_confusion_matrix(
    y_true=y_test,
    y_pred=optimal_ada_boost_y_pred
)

#add code here to add your own model 



model_cm_dict = {
    "Optimal DT": optimal_decision_tree_cm,
    "Optimal ADA": optimal_adaboost_cm
}

plot_multilabel_confusion_matrices(model_cm_dict)


A useful evaluation is looking at the Reciever Operating Characteristic (ROC)-curve. This shows how well a model's predicted probabilities separate positive from negative cases across all thresholds, with the area under the curve (AUC) summarizing this ability as a single number. 0.5 is considered to be random, while 1.0 is a perfect (with curve in the upper left corner). 

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

def plot_single_roc(y_true, y_score, label):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    auc = roc_auc_score(y_true, y_score)
    plt.plot(fpr, tpr, label=f"{label} (AUC = {auc:.3f})")

In [ ]:
#roc curve for H1N1 

plt.figure(figsize=(7, 7))

plot_single_roc(
    y_test["h1n1_vaccine"],
    optimal_decision_tree_y_preds["h1n1_vaccine"],
    label="Decision Tree"
)

plot_single_roc(
    y_test["h1n1_vaccine"],
    optimal_ada_boost_y_preds["h1n1_vaccine"],
    label="AdaBoost"
)


#add code here to add your own model 

# chance line
plt.plot([0, 1], [0, 1], linestyle="--", color="grey", linewidth=2)

plt.xlabel("False Positive Rate (FPR)", fontsize=14)
plt.ylabel("True Positive Rate (TPR)", fontsize=14)
plt.title("a) ROC Curve for H1N1 Vaccine", fontsize=16)

plt.xticks(np.arange(0.0, 1.1, 0.1))
plt.yticks(np.arange(0.0, 1.1, 0.1))

plt.legend()
plt.show()

In [ ]:
# add code here to plot a ROC curve for the seasonal vaccine for all tested models 

Reason around the confusion matrices and the ROC-curves for the two different labels and compare the models. 

* Which of the tested classification models is the most suitable to predict vaccination rate of the H1N1 and the Seasonal vaccine?
* Does one vaccine seem easier to predict than the other?
* If you were to continue the work to get a better performance, what could you do differently or add? 

